## Imports

In [1]:
import pandas as pd

## Leitura das bases da camada silver

In [2]:
df_insights = pd.read_parquet('C:/Users/candi/OneDrive/Documentos/GitHub/case_enjoei/datasets/gold/dataset_gold.parquet')

## Informações solicitadas pelo négocio

In [ ]:
# Agrupamento por usuário e categoria + soma das quantidades de produtos por categoria
category_totals = df_insights.groupby(['user_id', 'category'])['quantity'].sum().reset_index()

# Encontrando a categoria mais adicionada pelo usuário no carrinho
most_category = category_totals.loc[category_totals.groupby('user_id')['quantity'].idxmax()]

In [4]:
# Última data de adição ao carrinho por usuário
last_date_add = df_insights.groupby('user_id')['date'].max().reset_index()

# Merge para incorporar a última data ao dataframe
most_category = most_category.merge(last_date_add, on='user_id')

In [5]:
# Fazendo o merge para incorporar a coluna 'name' ao DataFrame
merge = most_category.merge(df_insights[['user_id', 'full_name']], on='user_id')

# Reorganizando as colunas para que 'full_name' venha após 'user_id'
insight = merge[['user_id', 'full_name', 'date', 'category', 'quantity']]

insight = insight.drop_duplicates()
insight

,user_id,full_name,date,category,quantity
0,1,john doe,2020-03-02,men's clothing,25
6,2,david morrison,2020-03-01,men's clothing,2
8,3,kevin ryan,2020-03-01,men's clothing,4
11,4,don romer,2020-03-01,electronics,5
13,8,william hopkins,2020-03-01,women's clothing,1


## Insights adicionais

### Top comprador por categoria

In [ ]:
# 1. Agrupando por 'category' e 'user_id' e somando a quantidade de produtos para cada usuário dentro de cada categoria
category_top_buyers = df_insights.groupby(['category', 'user_id', 'full_name'], as_index=False)['quantity'].sum()

# 2. Encontrando o "top comprador" em cada categoria
# Para cada categoria, selecionamos o usuário com a maior quantidade de produtos
top_buyers_per_category = category_top_buyers.loc[category_top_buyers.groupby('category')['quantity'].idxmax()]

# 3. Selecionando as colunas desejadas: 'user_id', 'full_name', 'quantity', e 'category'
top_buyers_per_category = top_buyers_per_category[['user_id', 'full_name', 'category', 'quantity']]

# Exibindo o resultado
top_buyers_per_category


,user_id,full_name,category,quantity
1,4,don romer,electronics,5
2,1,john doe,jewelery,2
4,1,john doe,men's clothing,25
7,8,william hopkins,women's clothing,1


### Ordem decrescente de itens no carrinho por úsuario

In [24]:
# 1. Agrupando por 'user_id' e somando a quantidade total de itens no carrinho para cada usuário
user_cart_totals = df_insights.groupby(['user_id', 'full_name'], as_index=False)['quantity'].sum()

# 2. Ordenando os usuários pelo total de itens no carrinho em ordem decrescente
user_cart_totals_sorted = user_cart_totals.sort_values(by='quantity', ascending=False).reset_index(drop=True)

# Exibindo o resultado
user_cart_totals_sorted



,user_id,full_name,quantity
0,1,john doe,27
1,3,kevin ryan,6
2,4,don romer,5
3,2,david morrison,3
4,8,william hopkins,1


### Produto mais caro por categoria

In [8]:
# 1. Para cada categoria, encontrando o índice do item com o preço mais alto
top_items_by_category = df_insights.loc[df_insights.groupby('category')['price'].idxmax()]

# 2. Selecionando colunas relevantes para visualização
top_items_by_category = top_items_by_category[['category', 'product_id', 'title', 'price', 'description']]

# Exibindo o resultado
top_items_by_category


,category,product_id,title,price,description
12,electronics,12,WD 4TB Gaming Drive Works with Playstation 4 P...,114.00,"Expand your PS4 gaming experience, Play anywhe..."
5,jewelery,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp..."
0,men's clothing,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...
13,women's clothing,18,MBJ Women's Solid Short Sleeve Boat Neck V,9.85,"95% RAYON 5% SPANDEX, Made in USA or Imported,..."


In [6]:
colunas_ordenadas = [
    'user_id',
    'full_name',
    'product_id',
    'category',
    'quantity',
    'price',
    'date',
    'carts_id'
]

dfy = df_insights[colunas_ordenadas]
dfy

,user_id,full_name,product_id,category,quantity,price,date,carts_id
0,1,john doe,1,men's clothing,4,109.95,2020-03-02,1
1,1,john doe,2,men's clothing,1,22.30,2020-03-02,1
2,1,john doe,3,men's clothing,6,55.99,2020-03-02,1
3,1,john doe,2,men's clothing,4,22.30,2020-01-02,2
4,1,john doe,1,men's clothing,10,109.95,2020-01-02,2
5,1,john doe,5,jewelery,2,695.00,2020-01-02,2
6,2,david morrison,1,men's clothing,2,109.95,2020-03-01,3
7,2,david morrison,9,electronics,1,64.00,2020-03-01,3
8,3,kevin ryan,1,men's clothing,4,109.95,2020-01-01,4
9,3,kevin ryan,7,jewelery,1,9.99,2020-03-01,5
